In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline


In [69]:
df = pd.read_csv("diabetes.csv")

In [70]:
df.sample(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
477,7,114,76,17,110,23.8,0.466,31,0
494,3,80,0,0,0,0.0,0.174,22,0
543,4,84,90,23,56,39.5,0.159,25,0
14,5,166,72,19,175,25.8,0.587,51,1
437,5,147,75,0,0,29.9,0.434,28,0


In [71]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


# training without binnig:

In [73]:
# x = df.iloc[:, 0:-1]
x = df.drop(columns="Outcome")

y= df.iloc[:, -1]


In [74]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=None)

In [75]:
dt = DecisionTreeClassifier()

In [76]:
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [77]:
y_pred = dt.predict(x_test)

In [78]:
print(f"Accuracy score is: {accuracy_score(y_pred, y_test)*100}")

Accuracy score is: 70.56277056277057


In [111]:
print(f"Mean accuracy score: {np.mean(cross_val_score(dt, x,y, cv=10))*100}")

Mean accuracy score: 69.91455912508545


# now binning and training:

In [80]:
trf1 = ColumnTransformer([
    ("Glucose", KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), ["Glucose"]),
    ("BMI", KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), ["BMI"]),
    ("Age", KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), ["Age"]),
    ("Insulin", KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), ["Insulin"])
],
    remainder="passthrough"
)

In [67]:
trf2 = DecisionTreeClassifier()

In [82]:
pipe = Pipeline([
    ("transformer", trf1),
    ("classifier", trf2)
])

In [83]:
pipe.fit(x_train, y_train)

/home/toqeer-yasir/anaconda3/lib/python3.12/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Glucose',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   n_bins=10,
                                                                   strategy='uniform'),
                                                  ['Glucose']),
                                                 ('BMI',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   n_bins=10,
                                                                   strategy='uniform'),
                                                  ['BMI']),
                                                 ('Age',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   n_bins=10,
                                                                   strategy='uniform'),
                                                  ['Age']),
                                                 ('Insulin',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   n_bins=10,
                                                                   strategy='uniform'),
                                                  ['Insulin'])])),
                ('classifier', DecisionTreeClassifier())])

In [119]:
y_pred = pipe.predict(x_test)

In [120]:
print(f"Accuracy score is: {accuracy_score(y_pred, y_test)*100}")

Accuracy score is: 68.3982683982684


# mean accuracy score:

In [125]:
print(f"Mean accuracy score: {np.mean(cross_val_score(dt, x,y, cv=10))*100}")

Mean accuracy score: 71.47812713602188
